#### Import libraries and Define Functions

In [ ]:
import os
import numpy as np
import bigfish
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
import bigfish.plot as plot
from matplotlib.colors import LinearSegmentedColormap
print("Big-FISH version: {0}".format(bigfish.__version__))

from scipy.ndimage import binary_dilation
from skimage.measure import label, regionprops

import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
import tifffile as tiff
import pandas as pd 
from skimage import io
import imageio
import matplotlib.patheffects as path_effects

from PIL import Image
import glob
from pathlib import Path
import pathlib
from tqdm import tqdm
import json

def find_spots_segment(image, mask, plot_result = False):
    """
    Find number of spots within each cell of an image
    :param image: numpy array of the image to be quantified
    :param mask: numpy array of the mask
    :param plot_result: whether to plot the results. If set to yes, every 50th cell will be plotted for checking
    :return: a list of spot count: one value per cell 
    """
    all_s = np.arange(1,mask.max()+1)
    spot_list = []
    areas = []
    for seg_num in tqdm(all_s, desc = 'Processing cells'):
        s = seg_num
        s_ind = np.where(mask == s)

        # Find segment coordinates (add 5 pixels from each side just to make sure all spots are picked up)
        min_x = np.max([0,s_ind[0].min() -5])
        min_y = np.max([0,s_ind[1].min() -5])
        max_x = np.min([mask.shape[0], s_ind[0].max()+5])
        max_y = np.min([mask.shape[1], s_ind[1].max()+5])

        segment_mask = mask[min_x:max_x, min_y:max_y]
        segment_image = image[min_x:max_x, min_y:max_y]

        #spot detection for single segment/cell
        spots = detection.detect_spots(images=segment_image, threshold=175, log_kernel_size=1.5, minimum_distance=1)
        spots = spots[segment_mask[spots[:,0],spots[:,1]]==s] # only choose spots that belong to the mask

        spot_list.append(spots)
        areas.append(len(s_ind[0]))
        if plot_result and seg_num%25 ==0:
            fig,[ax1,ax2] = plt.subplots(ncols = 2, figsize = (10,10))
            ax1.imshow(segment_image)
            ax1.imshow(segment_mask, cmap = 'gray',alpha = 0.3)
            ax2.scatter(spots[:,1],spots[:,0], alpha = 1,s=14, edgecolor = 'red',facecolors = 'none', linewidths=1.5, marker = 'o')
            ax2.imshow(segment_image)
            ax2.imshow(segment_mask,alpha = 0.1, cmap = 'gray')
            text2 = ax2.text(20, 10, f'{len(spots)} spots', fontsize=18, color='white', ha='center', va='center')
            text2.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'), path_effects.Normal()])

    return spot_list,areas



def count_rna(img_dir, mask_stain, plot_result = True, save_result = True):
    # make sure the directory is a Path object
    if type(img_dir) == str:
        img_dir = Path(img_dir)

    img_stain =img_dir.name.split('.tiff')[0].split('_stain-')[1]
    img_name = img_dir.name
    batch_name = img_name[(img_name.find('_batch-')+7):img_name.find('_stain-')]
    equivalent_mask = root_mask_dir/mask_stain/img_name.replace(f'_stain-{img_stain}.tiff',f'_stain-{mask_stain}.png')

    # Create a dictionary for the lowest acceptable number of pixels within a cluster. These values are determined manually
    # as the 3rd percentile of the cluster areas for the manually-annotated images
    threshold_dic = {'NF200':4867, 'P2RX3':700}
    lower_th = threshold_dic[mask_stain] if mask_stain in threshold_dic else 0

    lower_th = threshold_dic[mask_stain]
    if os.path.exists(equivalent_mask):
        grand_parent = img_dir.parents[3]
        sub_name, sub_sec = img_dir.name.split('_date')[0].split('_')
        result_fname = f'{img_stain}_{mask_stain}'
        file_fname = img_dir.name.replace('.tiff','.xlsx').replace(f'_stain-{img_stain}',f'_stain-{img_stain}_{mask_stain}') 
        result_dir = grand_parent/'analysis'/'results'/result_fname/(file_fname)

        

        if os.path.exists(result_dir) and not plot_result:
            print(f'Processing image: \033[1;34m{img_name}\033[0m\033[1;32m ... results found, skipping ...\033[0m')
        else:
            print(f"Processing image: \033[1;34m{img_name}\033[0m")
            try:
                # Split the image into clusters 
                image = np.array(Image.open(img_dir), dtype = 'uint16')
                mask = np.array(Image.open(equivalent_mask), dtype = 'uint16')
                mask = remove_small_areas(mask, lower_th)


                # make sure the results folder exists for that participant
                if not os.path.exists(result_dir.parent):
                    os.makedirs(result_dir.parent)

                spot_list, areas = find_spots_segment(image, mask, plot_result = plot_result)

                spot_count = [len(spots) for spots in spot_list]

                # Save results
                spots_df = pd.DataFrame({'subject':[sub_name],'section':[sub_sec], 'batch':[batch_name], 'image_stain':[img_stain], 'mask_stain':[mask_stain],'img_name':[img_name],
                                        'spot_counts':[spot_count],'number of clusters':[len(spot_count)], 'average count per cluster':[np.nanmean(spot_count)], 'total area':[np.nansum(areas)]})
                if save_result:
                    print('saving to',result_dir)
                    spots_df.to_excel(result_dir, index = False)
                return spots_df
            except:
                print(f"This image was not processed: \033[1;31m{img_name}\033[0m")
    else:
        print(f'Equivalent mask for the image \033[1;31m{img_name}\033[0m was not found')


def remove_small_areas(mask,lower_th):
    """
    remove clusters with areas lower than a particular threshold
    :param mask: mask with single labels per cluster (if you have values of 0 and 255), it will be considerd one big cluster)
    :param lower_th: lower accepted number of pixel count per cluster
    :return: mask without the areas lower than the assigned threshold
    """
    new_mask = mask.copy()
    areas = {region.label:region.area for region in regionprops(new_mask)}
    small_areas = np.array([label for label in areas.keys() if areas[label]<lower_th])
    new_mask[np.isin(new_mask,small_areas)] = 0
    return label(new_mask)





def generate_cmap(channel = 'NF200'):
    """
    generate the appropriate cmap, depending on the type of channel
    """
    # Define colors
    light_green = 'green'
    light_blue = 'blue'
    light_red = 'red'
    light_cyan = 'cyan'

    # Create colormaps
    green_cmap = LinearSegmentedColormap.from_list('green_cmap', ['black', light_green])
    blue_cmap = LinearSegmentedColormap.from_list('blue_cmap', ['black', light_blue])
    red_cmap = LinearSegmentedColormap.from_list('red_cmap', ['black', light_red])
    cyan_cmap = LinearSegmentedColormap.from_list('cyan_cmap', ['black', light_cyan])

    cmaps_dic = {'NF200':green_cmap,'FABP7':green_cmap, 'DAPI':blue_cmap, 'P2RX3':red_cmap, 'CIRL1': cyan_cmap, 'CIRL3': cyan_cmap}
    return cmaps_dic[channel]




def plot_mask(image_dir,mask_dir, img_stain, vmin =None,vmax = None,figsize = (10,10)):
    # def plot_contours(image_dir,mask_dir):
    image = np.array(Image.open(image_dir))
    mask = np.array(Image.open(mask_dir))
    mask[mask>0]=255
    mask = mask.astype('uint8')

    ratio = image.shape[0]/1500
    image = cv2.resize(image,(int(image.shape[0]/ratio),int(image.shape[1]/ratio)))
    mask = cv2.resize(mask,(int(mask.shape[0]/ratio),int(mask.shape[1]/ratio)))

    # Threshold the image to ensure it is binary (white clusters on a black background)
    # Using cv2.THRESH_BINARY or cv2.THRESH_BINARY_INV depending on the image's original format
    _, thresholded_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresholded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contour_mask = np.zeros_like(image)


    cv2.drawContours(contour_mask, contours, -1, (255, 255),4)

    aa = contour_mask.copy()
    aa[aa>0] =1
    aa[aa<1] = 0.2

    plt.subplots(figsize = figsize)
    if vmin or vmax:
        plt.imshow(image, cmap= generate_cmap(img_stain), vmin = vmin, vmax = vmax)
    else:
        plt.imshow(image, cmap= generate_cmap(img_stain))
    plt.imshow(contour_mask, alpha= aa, cmap='gray')
    plt.show()

Image.MAX_IMAGE_PIXELS = 400000000

### Here is how the code works
<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/refs/heads/main/NF_Fish.png" width=900 height=450 />

In [ ]:
root_img_dir = Path(r"E:\MASTER THESIS_MARIA\RNA Scope Project\primary")
root_mask_dir = Path(r"E:\MASTER THESIS_MARIA\RNA Scope Project\analysis\masks")

img_stain = 'CIRL1' # can be 'CIRL1' or 'CIRL3'
mask_stain = 'NF200' # can be 'P2RX3', 'FABP7 or 'NF200'


for img_dir in root_img_dir.rglob(f'*_stain-{img_stain}.tiff'):
    count_rna(img_dir, mask_stain, plot_result = True, save_result = True)


### Double-checking
It is a good idea to check the performance of your code. You can use the cell below to observe the outcome for some images. when 'plot_result' is set to True, representative images will be plotted for you to inspect. It is always good to double-check.

In [ ]:
root_img_dir = Path(r"E:\MASTER THESIS_MARIA\RNA Scope Project\primary")
root_mask_dir = Path(r"E:\MASTER THESIS_MARIA\RNA Scope Project\analysis\masks")

img_dir = Path(r'E:\MASTER THESIS_MARIA\RNA Scope Project\primary\NF200_DAPI_P2RX3_CIRL1\R3\sub-R3_sec-1_date-08_25_24_batch-NF200_DAPI_P2RX3_CIRL1_stain-CIRL1.tiff')
mask_stain = 'P2RX3' # can be 'P2RX3', 'FABP7 or 'NF200'



img_stain =img_dir.name.split('.tiff')[0].split('_stain-')[1]
mask_dir = root_mask_dir/mask_stain/img_dir.name.replace(f'_stain-{img_stain}.tiff',f'_stain-{mask_stain}.png')

plot_mask(img_dir,mask_dir, img_stain=img_stain, vmax = 1000, figsize=(15,15))

spot_list = count_rna(img_dir, mask_stain, plot_result = True)
spot_list



# Concatenate excel files
The files will be saved under the results directory, they will end with "_con_res.xlsx"

In [13]:
results_dir = Path(r"E:\MASTER THESIS_MARIA\RNA Scope Project\analysis\results\CIRL1_NF200")


excel_dir = results_dir.parent/(results_dir.name+'_con_res.xlsx')
all_data = pd.concat([pd.read_excel(fname) for fname in results_dir.rglob('*.xlsx')],axis = 0)

all_data.to_excel(excel_dir,index = False)